##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Recommending movies: ranking

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/recommenders/examples/basic_ranking"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/recommenders/blob/main/docs/examples/basic_ranking.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/recommenders/blob/main/docs/examples/basic_ranking.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/recommenders/docs/examples/basic_ranking.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>



Real-world recommender systems are often composed of two stages:

1. The retrieval stage is responsible for selecting an initial set of hundreds of candidates from all possible candidates. The main objective of this model is to efficiently weed out all candidates that the user is not interested in. Because the retrieval model may be dealing with millions of candidates, it has to be computationally efficient.
2. The ranking stage takes the outputs of the retrieval model and fine-tunes them to select the best possible handful of recommendations. Its task is to narrow down the set of items the user may be interested in to a shortlist of likely candidates.

We're going to focus on the second stage, ranking. If you are interested in the retrieval stage, have a look at our [retrieval](basic_retrieval) tutorial.

In this tutorial, we're going to:

1. Get our data and split it into a training and test set.
2. Implement a ranking model.
3. Fit and evaluate it.


## Imports


Let's first get our imports out of the way.

In [1]:
!pip install -q --upgrade tensorflow==2.3.0
!pip install -q --upgrade tensorflow-datasets

ERROR: tensorflow-serving-api 1.15.0 has requirement tensorflow~=1.15.0, but you'll have tensorflow 2.3.0 which is incompatible.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/tensorflow_p36/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/tensorflow_p36/bin/python -m pip install --upgrade pip' command.


In [4]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [5]:
import tensorflow_recommenders as tfrs

## Preparing the dataset

We're going to use the same data as the [retrieval](basic_retrieval) tutorial. This time, we're also going to keep the ratings: these are the objectives we are trying to predict.

In [6]:
ratings = tfds.load("movie_lens/100k-ratings", split="train")

ratings = ratings.map(lambda x: {
    "movie_title": x["movie_id"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"]
})

As before, we'll split the data by putting 80% of the ratings in the train set, and 20% in the test set.

In [7]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

Let's also figure out unique user ids and movie titles present in the data. 

This is important because we need to be able to map the raw values of our categorical features to embedding vectors in our models. To do that, we need a vocabulary that maps a raw feature value to an integer in a contiguous range: this allows us to look up the corresponding embeddings in our embedding tables.

In [8]:
movie_titles = ratings.batch(1_000_000).map(lambda x: x["movie_title"])
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

## Implementing a model

### Architecture

Ranking models do not face the same efficiency constrains as retrieval models do, and so we have a little bit more freedom in our choice of architectures.

A model composed of multiple stacked dense layers is a relatively common architecture for ranking tasks. We can implement it as follows:

In [9]:
class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32

    # Compute embeddings for users.
    self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # Compute embeddings for movies.
    self.movie_embeddings = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
  ])
    
  def call(self, user_id, movie_title):
    user_embedding = self.user_embeddings(user_id)
    movie_embedding = self.movie_embeddings(movie_title)
    return self.ratings(tf.concat([user_embedding, movie_embedding], axis=1))

This model takes user ids and movie titles, and outputs a predicted rating:

### Loss and metrics

The next component is the loss used to train our model. TFRS has several loss layers and tasks to make this easy.

In this instance, we'll make use of the `Ranking` task object: a convenience wrapper that bundles together the loss function and metric computation. 

We'll use it together with the `MeanSquaredError` Keras loss in order to predict the ratings.

In [10]:
task = tfrs.tasks.Ranking(
  loss = tf.keras.losses.MeanSquaredError(),
  metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

The task itself is a Keras layer that takes true and predicted as arguments, and returns the computed loss. We'll use that to implement the model's training loop.

### The full model

We can now put it all together into a model. TFRS exposes a base model class (`tfrs.models.Model`) which streamlines bulding models: all we need to do is to set up the components in the `__init__` method, and implement the `compute_loss` method, taking in the raw features and returning a loss value.

The base model will then take care of creating the appropriate training loop to fit our model.

In [11]:
class MovielensModel(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel()
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    rating_predictions = self.ranking_model(
        features["user_id"], features["movie_title"])
    print('rating_predictions', rating_predictions)
    # The task computes the loss and the metrics.
    return self.task(labels=features["user_rating"], predictions=rating_predictions)

## Fitting and evaluating

After defining the model, we can use standard Keras fitting and evaluation routines to fit and evaluate the model.

Let's first instantiate the model.

In [12]:
model = MovielensModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

Then shuffle, batch, and cache the training and evaluation data.

In [13]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

Then train the  model:

In [14]:
model.fit(cached_train, epochs=100)

Epoch 1/100
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
rating_predictions Tensor("ranking_model/sequential_2/dense_2/BiasAdd:0", shape=(None, 1), dtype=float32)
rating_predictions Tensor("ranking_model/sequential_2/dense_2/BiasAdd:0", shape=(None, 1), dtype=float32)
10/10 [==============================] - 0s 38ms/step - root_mean_squared_error: 2.1105 - loss: 4.1010 - regularization_loss: 0.0000e+00 - total_loss: 4.1010
Epoch 2/100
10/10 [==============================] - 0s 24ms/step - root_mean_squared_error: 1.1293 - loss: 1.2780 - regularization_loss: 0.0000e+00 - total_loss: 1.2780
Epoch 3/100
10/10 [==============================] - 0s 24ms/step - root_mean_squared_error: 1.1222 - loss: 1.2614 - 

10/10 [==============================] - 0s 24ms/step - root_mean_squared_error: 0.9181 - loss: 0.8459 - regularization_loss: 0.0000e+00 - total_loss: 0.8459
Epoch 45/100
10/10 [==============================] - 0s 24ms/step - root_mean_squared_error: 0.9179 - loss: 0.8455 - regularization_loss: 0.0000e+00 - total_loss: 0.8455
Epoch 46/100
10/10 [==============================] - 0s 24ms/step - root_mean_squared_error: 0.9177 - loss: 0.8452 - regularization_loss: 0.0000e+00 - total_loss: 0.8452
Epoch 47/100
10/10 [==============================] - 0s 24ms/step - root_mean_squared_error: 0.9176 - loss: 0.8449 - regularization_loss: 0.0000e+00 - total_loss: 0.8449
Epoch 48/100
10/10 [==============================] - 0s 24ms/step - root_mean_squared_error: 0.9174 - loss: 0.8447 - regularization_loss: 0.0000e+00 - total_loss: 0.8447
Epoch 49/100
10/10 [==============================] - 0s 24ms/step - root_mean_squared_error: 0.9173 - loss: 0.8444 - regularization_loss: 0.0000e+00 - total_

Epoch 92/100
10/10 [==============================] - 0s 24ms/step - root_mean_squared_error: 0.9139 - loss: 0.8378 - regularization_loss: 0.0000e+00 - total_loss: 0.8378
Epoch 93/100
10/10 [==============================] - 0s 24ms/step - root_mean_squared_error: 0.9138 - loss: 0.8377 - regularization_loss: 0.0000e+00 - total_loss: 0.8377
Epoch 94/100
10/10 [==============================] - 0s 25ms/step - root_mean_squared_error: 0.9138 - loss: 0.8376 - regularization_loss: 0.0000e+00 - total_loss: 0.8376
Epoch 95/100
10/10 [==============================] - 0s 25ms/step - root_mean_squared_error: 0.9137 - loss: 0.8374 - regularization_loss: 0.0000e+00 - total_loss: 0.8374
Epoch 96/100
10/10 [==============================] - 0s 24ms/step - root_mean_squared_error: 0.9136 - loss: 0.8373 - regularization_loss: 0.0000e+00 - total_loss: 0.8373
Epoch 97/100
10/10 [==============================] - 0s 24ms/step - root_mean_squared_error: 0.9135 - loss: 0.8372 - regularization_loss: 0.0000

As the model trains, the loss is falling and the RMSE metric is improving.

Finally, we can evaluate our model on the test set:

In [15]:
model.evaluate(cached_test, return_dict=True)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
rating_predictions Tensor("ranking_model/sequential_2/dense_2/BiasAdd:0", shape=(None, 1), dtype=float32)
5/5 [==============================] - 0s 13ms/step - root_mean_squared_error: 0.9365 - loss: 0.8784 - regularization_loss: 0.0000e+00 - total_loss: 0.8784


{'root_mean_squared_error': 0.9365375638008118,
 'loss': 0.8838098645210266,
 'regularization_loss': 0,
 'total_loss': 0.8838098645210266}

In [16]:
cached_test
# print(type(model.ranking_model))
model.ranking_model(tf.convert_to_tensor(["42"]),tf.convert_to_tensor(["80"]))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[3.104019]], dtype=float32)>

In [50]:
import requests 

# r = requests.get('https://search-p13n-search-demo-g5k65bmwi3rlvipzvlpeugem3m.us-east-1.es.amazonaws.com/movies/movie/_search', json=query, auth=('yianc', 'Cj;6qo4fu6@@0218'))
r = requests.get('https://search-p13n-search-demo-g5k65bmwi3rlvipzvlpeugem3m.us-east-1.es.amazonaws.com/movies/movie/_search?q=Drama:1&size=100', auth=('yianc', 'Cj;6qo4fu6@@0218'))
rjson = r.json()
rjson 

{'took': 22,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1453, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'movies',
    '_type': 'movie',
    '_id': 'cXTWAHUB_7LrTuSkMhgU',
    '_score': 1.0,
    '_source': {'iid': 18,
     'title': 'White Balloon, The (1995)',
     'release_date': '01-Jan-1995',
     'video_release_date': '',
     'imdb url': 'http://us.imdb.com/M/title-exact?Badkonake%20Sefid%20(1995)',
     'unknown': 0,
     'Action': 0,
     'Adventure': 0,
     'Animation': 0,
     'Childrens': 0,
     'Comedy': 0,
     'Crime': 0,
     'Documentary': 0,
     'Drama': 1,
     'Fantasy': 0,
     'Film-Noir': 0,
     'Horror': 0,
     'Musical': 0,
     'Mystery': 0,
     'Romance': 0,
     'Sci-Fi': 0,
     'Thriller': 0,
     'War': 0,
     'Western': 0,
     'year': 1995}},
   {'_index': 'movies',
    '_type': 'movie',
    '_id': 'UxrWAHUBGY5JWo_sObSL',
    '_score': 1.0,
    '_source':

In [51]:
import pandas as pd 
data_dir="~/SageMaker/20201020-dnb-ranking-optimization/ml-100k"
users_df = pd.read_csv(data_dir + '/u.user', names=['uid','age','gender','occupation','zipcode'],  sep='|')
users_df.head()

,uid,age,gender,occupation,zipcode
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [70]:
items_from_search = [] 
for h in rjson['hits']['hits']:
    items_from_search.append(h['_source']['iid'])

rerank_user = 244

In [71]:
original_data = pd.read_csv(data_dir + '/u.data', delimiter='\t', header=None)
original_data = original_data.rename(columns={0: "user", 1: "item", 2:"rating", 3:"timestamp"})
original_data.head(5)

,user,item,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [72]:
interactions_df = original_data.copy()
interactions_df = interactions_df[['user', 'item', 'rating', 'timestamp']]
interactions_df.head()

,user,item,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [73]:
import requests 

# r = requests.get('https://search-p13n-search-demo-g5k65bmwi3rlvipzvlpeugem3m.us-east-1.es.amazonaws.com/movies/movie/_search', json=query, auth=('yianc', 'Cj;6qo4fu6@@0218'))
r = requests.get('https://search-p13n-search-demo-g5k65bmwi3rlvipzvlpeugem3m.us-east-1.es.amazonaws.com/movies/movie/_search?q=Drama:1&size=100', auth=('yianc', 'Cj;6qo4fu6@@0218'))
rjson = r.json()
rjson 


{'took': 16,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1453, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'movies',
    '_type': 'movie',
    '_id': 'cXTWAHUB_7LrTuSkMhgU',
    '_score': 1.0,
    '_source': {'iid': 18,
     'title': 'White Balloon, The (1995)',
     'release_date': '01-Jan-1995',
     'video_release_date': '',
     'imdb url': 'http://us.imdb.com/M/title-exact?Badkonake%20Sefid%20(1995)',
     'unknown': 0,
     'Action': 0,
     'Adventure': 0,
     'Animation': 0,
     'Childrens': 0,
     'Comedy': 0,
     'Crime': 0,
     'Documentary': 0,
     'Drama': 1,
     'Fantasy': 0,
     'Film-Noir': 0,
     'Horror': 0,
     'Musical': 0,
     'Mystery': 0,
     'Romance': 0,
     'Sci-Fi': 0,
     'Thriller': 0,
     'War': 0,
     'Western': 0,
     'year': 1995}},
   {'_index': 'movies',
    '_type': 'movie',
    '_id': 'UxrWAHUBGY5JWo_sObSL',
    '_score': 1.0,
    '_source':

In [74]:
items_from_search = [] 
for h in rjson['hits']['hits']:
    items_from_search.append(h['_source']['iid'])

rerank_user = users_df.sample(1).index.tolist()[0]

In [75]:
# Create a dataframe for the items by reading in the correct source CSV
genres = ['unknown','Action' , 'Adventure', 'Animation', 'Childrens' , 'Comedy' , 'Crime', \
                                        'Documentary', 'Drama' ,'Fantasy' , 'Film-Noir' , 'Horror' , 'Musical', \
                                        'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items_df = pd.read_csv(data_dir + '/u.item', names=['iid','title','release_date','video_release_date', 'imdb url'] + genres,  sep='|', encoding = "ISO-8859-1")
# Render some sample data
items_df.head(5)

,iid,title,release_date,video_release_date,imdb url,unknown,Action,Adventure,Animation,Childrens,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [76]:
def get_movie_by_id(movie_id, movie_df=items_df):
    """
    This takes in an artist_id from Personalize so it will be a string,
    converts it to an int, and then does a lookup in a default or specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return movie_df[movie_df['iid']==movie_id].iloc[0]['title']
    except:
        return "Error obtaining artist"

In [77]:
rerank_list = []
for item in items_from_search:
    movie = get_movie_by_id(item)
    rerank_list.append(movie)
rerank_df = pd.DataFrame(rerank_list, columns = [rerank_user])
rerank_df

,233
0,"White Balloon, The (1995)"
1,Copycat (1995)
2,Shanghai Triad (Yao a yao yao dao waipo qiao) ...
3,Babe (1995)
4,Mr. Holland's Opus (1995)
...,...
95,"Inkwell, The (1994)"
96,Prefontaine (1997)
97,Kim (1950)
98,Kiss of Death (1995)


In [78]:
import numpy 
ranked_list = []
for item in items_from_search: 
    movie = get_movie_by_id(item)
    score = model.ranking_model(tf.convert_to_tensor([str(rerank_user)]),tf.convert_to_tensor([str(item)]))
    print([movie, score.numpy()[0][0]]) 
    ranked_list.append([movie, score.numpy()[0][0]])

ranked_list = sorted(ranked_list, key=lambda x:x[1], reverse=True)   
ranked_df = pd.DataFrame(numpy.array(ranked_list)[:,0], columns = ['Re-Ranked'])
all_df = pd.concat([rerank_df, ranked_df], axis=1)
all_df

['White Balloon, The (1995)', 3.8899517]
['Copycat (1995)', 3.859411]
['Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)', 4.3575263]
['Babe (1995)', 4.5457387]
["Mr. Holland's Opus (1995)", 4.238854]
['Angels and Insects (1995)', 4.0547204]
['Dolores Claiborne (1994)', 3.9427536]
['Eat Drink Man Woman (1994)', 4.7060385]
['Exotica (1994)', 4.258676]
['Priest (1994)', 4.7561235]
['Searching for Bobby Fischer (1993)', 4.4903936]
['Spitfire Grill, The (1996)', 4.314508]
['Bound (1996)', 4.3602467]
['Citizen Kane (1941)', 4.846261]
['On Golden Pond (1981)', 4.183256]
['Empire Strikes Back, The (1980)', 4.7476935]
['Full Metal Jacket (1987)', 4.355679]
['Bridge on the River Kwai, The (1957)', 4.7483807]
['Unbearable Lightness of Being, The (1988)', 4.097037]
['Field of Dreams (1989)', 4.2384357]
['Sneakers (1992)', 4.1631894]
['Evita (1996)', 3.5712967]
["Ulee's Gold (1997)", 4.3145576]
['Titanic (1997)', 4.7663193]
['Apt Pupil (1998)', 4.6479254]
['Cop Land (1997)', 3.8832626]
['Boogi

,233,Re-Ranked
0,"White Balloon, The (1995)",Some Folks Call It a Sling Blade (1993)
1,Copycat (1995),One Flew Over the Cuckoo's Nest (1975)
2,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,Citizen Kane (1941)
3,Babe (1995),Four Days in September (1997)
4,Mr. Holland's Opus (1995),Titanic (1997)
...,...,...
95,"Inkwell, The (1994)",Flesh and Bone (1993)
96,Prefontaine (1997),Poetic Justice (1993)
97,Kim (1950),Bent (1997)
98,Kiss of Death (1995),Nil By Mouth (1997)


The lower the RMSE metric, the more accurate our model is at predicting ratings.

## Next steps

The model above gives us a decent start towards building a ranking system.

Of course, making a practical ranking system requires much more effort.

In most cases, a ranking model can be substantially improved by using more features rather than just user and candidate identifiers. To see how to do that, have a look at the [side features](fearutization) tutorial.

A careful understanding of the objectives worth optimizing is also necessary. To get started on building a recommender that optimizes multiple objectives, have a look at our [multitask](multitask) tutorial.